In [35]:
import random

In [36]:
print(random.random())

0.10087757004439724


In [37]:
ObjLst = []

for i in range(100):
    a = random.random()*100
    b = random.random()*100
    ObjLst.append([a,b])

for obj in ObjLst[0:10]:
    print(obj)

[97.86798685226985, 9.496357969487136]
[21.467293989905567, 75.51947579865205]
[79.97890948429423, 37.150300715739036]
[49.291604440735085, 27.861939896666478]
[55.30184043496652, 51.807350965919206]
[11.086935799929865, 87.40398767497338]
[24.124168520170052, 1.035082004605703]
[19.115899044784157, 59.144352956121516]
[78.96022102220428, 50.61502017242996]
[4.54088122996763, 61.16398669069417]


In [38]:
from rtree import RTree, Rect

In [44]:
BR = Rect(0.0,0.0,10.0,10.0)

In [45]:
t = RTree()
i=1
for obj in ObjLst:
    print(obj)
    print(i)
    i=i+1
    t.insert(obj, BR)

[97.86798685226985, 9.496357969487136]
1
[21.467293989905567, 75.51947579865205]
2
[79.97890948429423, 37.150300715739036]
3
[49.291604440735085, 27.861939896666478]
4
[55.30184043496652, 51.807350965919206]
5
[11.086935799929865, 87.40398767497338]
6
[24.124168520170052, 1.035082004605703]
7
[19.115899044784157, 59.144352956121516]
8
[78.96022102220428, 50.61502017242996]
9
[4.54088122996763, 61.16398669069417]
10
[71.48325820531181, 55.64276336671015]
11
[93.29697540050572, 8.057900909993965]
12
[19.15030010826102, 57.52783589258831]
13
[45.495980264585945, 36.309175582179364]
14
[32.86474780562559, 63.99208696258592]
15
[54.047027440516295, 49.692940355663204]
16
[96.99445442026429, 67.39651907321029]
17
[79.21729731252849, 94.28302141701322]
18
[84.43201338726813, 23.733971760151185]
19
[42.82765300870116, 27.35710946561325]
20
[13.158597905933423, 63.84616674301014]
21
[74.31269397573153, 65.0468073196277]
22
[34.60478488868802, 93.09046955471288]
23
[81.63981020087003, 96.5308512

In [46]:
rect_res = t.query_rect(Rect(3.0,3.0,4.0,4.0))
real_rect_res = [r.leaf_obj() for r in rect_res if r.is_leaf()]

print(len(real_rect_res))

100


In [47]:
t.stats

{'overflow_f': 90,
 'avg_overflow_t_f': 0.0015116797553168394,
 'longest_overflow': 0.009647369384765625,
 'longest_kmeans': 0.007418394088745117,
 'sum_kmeans_iter_f': 540,
 'count_kmeans_iter_f': 270,
 'avg_kmeans_iter_f': 2.0}

In [48]:
t.leaf_count

100

In [49]:
t.node_pool

array('L', [0, 12, 23, 0, 121, 1, 173, 2, 171, 3, 6, 4, 19, 5, 163, 6, 189, 7, 185, 8, 117, 9, 147, 10, 0, 14, 81, 11, 0, 16, 157, 12, 0, 18, 13, 13, 0, 20, 67, 14, 0, 22, 10, 15, 0, 24, 73, 16, 0, 26, 55, 17, 0, 28, 79, 18, 0, 30, 169, 19, 0, 32, 51, 20, 0, 34, 43, 21, 0, 36, 111, 22, 0, 38, 59, 23, 0, 40, 135, 24, 0, 42, 5, 25, 0, 44, 37, 26, 0, 46, 99, 27, 0, 48, 131, 28, 0, 50, 161, 29, 0, 52, 87, 30, 0, 54, 155, 31, 0, 56, 153, 32, 0, 58, 151, 33, 0, 60, 159, 34, 0, 62, 2, 35, 0, 64, 149, 36, 0, 66, 181, 37, 0, 68, 107, 38, 0, 70, 113, 39, 0, 72, 103, 40, 0, 74, 35, 41, 0, 76, 167, 42, 0, 78, 139, 43, 0, 80, 97, 44, 0, 82, 75, 45, 0, 84, 39, 46, 0, 86, 177, 47, 0, 88, 61, 48, 0, 90, 69, 49, 0, 92, 29, 50, 0, 94, 183, 51, 0, 96, 25, 52, 0, 98, 175, 53, 0, 100, 27, 54, 0, 102, 45, 55, 0, 104, 115, 56, 0, 106, 91, 57, 0, 108, 15, 58, 0, 110, 9, 59, 0, 112, 165, 60, 0, 114, 3, 61, 0, 116, 31, 62, 0, 118, 141, 63, 0, 120, 127, 64, 0, 122, 83, 65, 0, 124, 109, 66, 0, 126, 143, 67, 0, 12

In [ ]:
def k_means_cluster(root, k, nodes):
    t = time.time()
    if len(nodes) <= k: return [ [n] for n in nodes ]

    ns = list(nodes)
    root.stats["count_kmeans_iter_f"] += 1

    # Initialize: take n random nodes.
    random.shuffle(ns)

    cluster_starts = ns[:k]
    cluster_centers = [ center_of_gravity([n]) for n in ns[:k] ]


    # Loop until stable:
    while True:
        root.stats["sum_kmeans_iter_f"] += 1
        clusters = [ [] for c in cluster_centers ]

        for n in ns:
            idx = closest(cluster_centers, n)
            clusters[idx].append(n)

        #FIXME HACK TODO: is it okay for there to be empty clusters?
        clusters = [ c for c in clusters if len(c) > 0 ]

        for c in clusters:
            if (len(c) == 0):
                print("Errorrr....")
                print("Nodes: %d, centers: %s" % (len(ns),
                                                              repr(cluster_centers)))

            assert(len(c) > 0)

        rest = ns
        first = False

        new_cluster_centers = [ center_of_gravity(c) for c in clusters ]
        if new_cluster_centers == cluster_centers :
            root.stats["avg_kmeans_iter_f"] = float(root.stats["sum_kmeans_iter_f"] / root.stats["count_kmeans_iter_f"])
            root.stats["longest_kmeans"] = max(root.stats["longest_kmeans"], (time.time() - t))
            return clusters
        else: cluster_centers = new_cluster_centers